In [1]:
#%pip uninstall stable-baselines3
#%pip install stable-baselines3==1.7.0 --no-deps

In [2]:
import os
from stable_baselines3 import PPO
from stable_baselines3 import DQN
from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv, VecTransposeImage
from matplotlib import pyplot as plt
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT,COMPLEX_MOVEMENT,RIGHT_ONLY
import torch

In [3]:
def createEnvironments_DQN(env_name):
    env = gym_super_mario_bros.make(env_name)
    env = JoypadSpace(env, SIMPLE_MOVEMENT)
    env = GrayScaleObservation(env, keep_dim=True)
    env = DummyVecEnv([lambda: env])
    env = VecFrameStack(env,4,channels_order='last')
    env = VecTransposeImage(env)
    return env

In [4]:
def createEnvironment_PPO(env_name):
        env = gym_super_mario_bros.make(env_name)
        env = JoypadSpace(env, SIMPLE_MOVEMENT)
        env = GrayScaleObservation(env, keep_dim=True)
        return env
    
def allWorldStages():
    worlds = [1, 2, 3, 4, 5, 6, 7, 8]
    #Training on Stages 1-3 for each world, testing on final stage of each world
    stages = [4]

    for world in worlds:
        for stage in stages:
            environment_name = f"SuperMarioBros-{world}-{stage}-v0"
            yield environment_name

In [5]:
env=createEnvironments_DQN("SuperMarioBros-4-4-v0")
observation_space = env.observation_space
action_space = env.action_space
device = torch.device('cuda')
model = DQN.load("./train/FinalModels/DQN10",env,device=device,custom_objects={'observation_space': observation_space, 'action_space': action_space})

c:\Users\nicho\anaconda3\envs\aci\lib\site-packages\gym\envs\registration.py:593: UserWarning: WARN: The environment SuperMarioBros-4-4-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
c:\Users\nicho\anaconda3\envs\aci\lib\site-packages\gym\core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\nicho\anaconda3\envs\aci\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\nicho\anaconda3\envs\aci\lib\site-packages\stable_baselines3\common\buffers.py:229: UserWarning: This system does not have apparently enough memory 

In [6]:
state = env.reset()
episode_reward = 0
episode_score = 0
episode_global_dist = 0
cum_reward = 0
count =0
while count <= 100:
    episode_global_dist = 0
    action, _state, = model.predict(state)
    state, reward, done, info = env.step(action)
    episode_reward =+ reward
    game_info = info[0]
    episode_score =+ game_info['score']
    episode_dist =+ game_info['x_pos']
    #if episode_dist > 500:
    #            episode_furthest_distance = max(episode_global_dist, episode_dist)
    #            episode_global_dist = max(episode_global_dist, episode_dist)
    #            print("TESTING MAX DIST")
    #            print("Cumulative Reward" , cum_reward)
    #            print(f"Reward {episode_reward}. Score {episode_score}. Distance {episode_global_dist}")
    #            print("-------------------------------")
    if episode_dist is not None:       
        episode_furthest_distance = max(episode_global_dist, episode_dist)
        episode_global_dist = max(episode_global_dist, episode_dist)
    cum_reward += reward
    if any(done):
        count = count + 1
        print("Episode" , count)
        print("Cumulative Reward" , cum_reward)
        print(f"Reward {episode_reward}. Score {episode_score}. Distance {episode_global_dist}")
    env.render()
env.close()

c:\Users\nicho\anaconda3\envs\aci\lib\site-packages\gym\utils\passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
c:\Users\nicho\anaconda3\envs\aci\lib\site-packages\gym\core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
c:\Users\nicho\anaconda3\envs\aci\lib\site-packages\gym\utils\passive_env_checker.py:280: UserWarning: WARN: No render modes was declared in the environment (env.metadata['render_modes'] is None or not defined), you may have trouble when calling `.render()`.
  logger.warn(


Episode 1
Cumulative Reward [142.]
Reward [-15.]. Score 0. Distance 204
Episode 2
Cumulative Reward [219.]
Reward [-15.]. Score 0. Distance 137
Episode 3
Cumulative Reward [296.]
Reward [-15.]. Score 0. Distance 137
Episode 4
Cumulative Reward [370.]
Reward [-15.]. Score 0. Distance 134
Episode 5
Cumulative Reward [444.]
Reward [-15.]. Score 0. Distance 134
Episode 6
Cumulative Reward [521.]
Reward [-15.]. Score 0. Distance 137
Episode 7
Cumulative Reward [595.]
Reward [-15.]. Score 0. Distance 134
Episode 8
Cumulative Reward [672.]
Reward [-15.]. Score 0. Distance 137
Episode 9
Cumulative Reward [749.]
Reward [-15.]. Score 0. Distance 137
Episode 10
Cumulative Reward [826.]
Reward [-15.]. Score 0. Distance 137
Episode 11
Cumulative Reward [965.]
Reward [-15.]. Score 0. Distance 200
Episode 12
Cumulative Reward [1040.]
Reward [-15.]. Score 0. Distance 135
Episode 13
Cumulative Reward [1113.]
Reward [-15.]. Score 0. Distance 133
Episode 14
Cumulative Reward [1190.]
Reward [-15.]. Score 

c:\Users\nicho\anaconda3\envs\aci\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


KeyboardInterrupt: 

: 